In [1]:
import numpy as np
from numpy import linalg as LA
import scipy.stats as scs
import matplotlib.pyplot as plt

In [108]:
def Kfun(d, norm = scs.norm(0,1)):
    #input: d real positive numbe or -1, and a normmal curve
    #output K: the integral between -d and d under the curve, unless d = -1
    # d=-1 is signal to return K = 1
    
    if d == -1:
        return(1.0)
    K = norm.cdf(d) - norm.cdf(-d)
    return(K)
def pcfun(d, norm = scs.norm(0,1)):
    #input: d real positive numbe or -1, and a normmal curve
    #output pc: the integral from d to infinity under the curve, unless d = -1
    # d=-1 is signal to return pc = 0
    if d == -1:
        return(0)
    pc = 1 - norm.cdf(d)
    return(pc)
def pwfun(d,norm = scs.norm(0,1)):
    #input: d real positive numbe or -1, and a normmal curve
    #output pw: the integral from d to infinity under the curve, unless d = -1
    # d=-1 is signal to return pw = 0
    if d == -1:
        return(0)
    pw = norm.cdf(-d)
    return(pw)

def phi_fun(p_i):
    #evaluated probability of learning a trait socially if there's conformist bias
    return(p_i*(1-p_i)*(2*p_i - 1))

def Wv_fun(p_i,v,r_i,D,K,pc):
    #v is a stand in for u, x, or y
    soc_learn = K*(p_i + D*phi_fun(p_i))
    to_return = v*(1+r_i)*(soc_learn + pc)
    return(to_return)
def Wvbar_fun(p_1,p_2,v,D,K,pc):
    pw = 1 - K - pc
    soc_learn = K*(1 - p_1 - p_2 - D*phi_fun(p_1 + p_2))
    to_return = v*(soc_learn + pw)
    return(to_return)

def ri_fun(r_i,p_i,gamma = 1, beta = 1):
    alpha = 1 + gamma
    ri_next = r_i*(alpha - beta*p_i*r_i)/(1+gamma*r_i)
    return(ri_next)

def NextGen(uvec,xvec,yvec,rvec, D, K,pc,deltas = [0, 0, 0], beta=1, gamma=1):
    #@inputs: curr is current values of u_1,u_2,bu,...
    #         The other inputs are the parameters
    #         deltas are [delta_D, delta_K, delta_pc]
    D_y = D + deltas[0]; K_x = K + deltas[1]; pc_x = pc + deltas[2];
    u = sum(uvec)
    x = sum(xvec)
    y = sum(yvec)
    p1 = uvec[0] + xvec[0] + yvec[0]
    p2 = uvec[1] + xvec[1] + yvec[1]
    Wu1 = Wv_fun(p1,u,rvec[0],D,K,pc)
    Wu2 = Wv_fun(p2,u,rvec[1],D,K,pc)
    Wbu = Wvbar_fun(p1,p2,u,D,K,pc)
    if x>0 and y >0:
        Wx1 = Wv_fun(p1,x,rvec[0],D,K_x,pc_x)
        Wx2 = Wv_fun(p2,x,rvec[1],D,K_x,pc_x)
        Wbx = Wvbar_fun(p1,p2,x,D,K_x,pc_x)
        Wy1 = Wv_fun(p1,y,rvec[0],D_y,K,pc)
        Wy2 = Wv_fun(p2,y,rvec[1],D_y,K,pc)
        Wby = Wvbar_fun(p1,p2,y,D,K,pc)
    else:
        Wx1 = Wx2 = Wbx = Wy1 = Wy2 = Wby = 0
    
    W = Wu1 + Wu2 + Wbu + Wx1 + Wx2 + Wbx + Wy1 + Wy2 + Wby
    freqs = (1/W)*np.array([Wu1, Wu2, Wbu, Wx1, Wx2, Wbx, Wy1, Wy2, Wby])
    uvec = freqs[0:3]; xvec = freqs[3:6]; yvec = freqs[6:9]
    
    rvec = [ri_fun(rvec[0], p1, gamma,beta), ri_fun(rvec[1],p2,gamma,beta)]
    return(uvec, xvec, yvec, rvec,W)

# helper functions for jacobian
def theta_fun(p):
    # derived from phi(p + delta_p) = phi(p) + delta_p theta(phi)
    # helper function for jacobian
    return(6*p*(1-p) - 1)

def Ai_jac(uvec,rvec,W,i):
    # given equilibrium values of u, r, W. i says which u, r to use. i = 0 or 1
    # output: a common constant in jacobian
    if i in [0,1]:
        Ai = W*uvec[i]/(1 + rvec[i])
        return(Ai)
    else:
        print("i out of range")
def Bi_jac(uvec,rvec,W,K,D,i):
    # given parameters K,D and equilibrium values uvec, r. i tells us which uvec, rvec entry to use
    # output: a common constant in jacobian
    ui = uvec[i]; ri = rvec[i]
    inside = 1 + D*theta_fun(ui) - D*theta_fun(sum(uvec)) + ri*(1 + D*phi_fun(ui))
    Bi = K*inside
    return(Bi)
# def C_jac(uvec, rvec, deltas, D):
#     # jacobian helper fun
#     # input: vector of u's, vector of r's, deltas to change params, and D
#     #         deltas are [delta_D, delta_K, delta_pc]
#     dd = deltas[0]
#     dk = deltas[1]
#     dpc = deltas[2]
#     u1 = uvec[0]; u2 = uvec[1];
#     r1 = rvec[0]; r2 = rvec[1];
#     C = dk*D*(phi_fun(u1) + phi_fun(u2) -phi_fun(u1+u2))#term 1
#     C = C + dpc
#     C = C + sum([rvec[i]*(dk*(uvec[i] + D*phi_fun(uvec[i]))+dpc) for i in [0,1]])   
#     return(C)
# def E_jac(uvec, rvec, deltas, K):
#     # jacobian helper fun
#     # input: vector of u's, vector of r's, deltas to change params, and D
#     #         deltas are [delta_D, delta_K, delta_pc]
#     dd = deltas[0]
#     u1 = uvec[0]; u2 = uvec[1]; r1 = rvec[0]; r2 = rvec[1];
#     E = K*dd[phi_fun(u1)*(1 + dd*r1) + phi_fun(u2)*(1+dd*r2) - phi(u1 + u2)]
#     return(E)

def U_jac(uvec, rvec, K, D,i):
    # gives U constant for Ui (if i = 0,1) or Ubar (if i = 2)
    if i in [0,1]:
        ui = uvec[i]; ri = rvec[i]
        U = (1 + ri)*K*(1 + D*phi_fun(ui))
    elif i == 2:
        U = K*D*theta_fun(sum(uvec))
    return(U)

def Jac_UR(uvec, rvec, K, D, W, deltas, beta, gamma):
    u1 = uvec[0]; u2 = uvec[1]; bu = uvec[2]; r1 = rvec[0]; r2 = rvec[1]
    Uv = [ U_jac(uvec, rvec, K, D, i) for i in [0,1,2]]
    Av = [Ai_jac(uvec, rvec, W, i) for i in [0,1]]
    Bv = [Bi_jac(uvec,rvec,W,K,D,i) for i in [0,1]]
    U1 = Uv[0]; 
    U2 = Uv[1]; 
    bU = Uv[2]; 
    A1 = Av[0]; 
    A2 = Av[1]; 
    B1 = Bv[0]; 
    B2 = Bv[1]
    Rpv = [ -beta*rvec[i]/(1+gamma*rvec[i]) for i in [0,1]]; 
    Rv = [1 - gamma*rvec[i]/(1+gamma*rvec[i]) for i in [0,1]]
    
    row1 = np.array([U1 + B1*u1, u1*B2, 0, A1*(1+u1), A2*u1])
    row1 = row1/(W - W*u1)
    row2 = np.array([u2*B1, U2 + B2*u2, 0, u2*A1, A2*(1+u2)])
    row2 = row2/(W - W*u2)
    row3 = np.array([-bU + bu*B1, -bU + bu*B2,0,bu*A1, bu*A2])
    row4 =[Rpv[0],0,0,Rv[0],0]
    row5 = [0,Rpv[0],0,0,Rv[1]]
    J = np.array([row1,row2,row3,row4,row5])
    return(J)

def eval_x(uvec, rvec, W, D, deltas):
    
    dk = deltas[1]; dpc = deltas[2]
    u1 = uvec[0]
    u2 = uvec[1]
    r1 = rvec[0]
    r2 = rvec[1]
    X1 = (1 + r1)*(dk*(u1 + D*phi_fun(u1)) + dpc)
    X2 = (1 + r1)*(dk*(u1 + D*phi_fun(u1)) + dpc)
    bX = -dk*(sum(uvec) + D*phi_fun(sum(uvec))) - dpc
    eval_x = 1 + (X1 + X2+ bX)/W
    return(eval_x)

def eval_y(uvec, rvec, W, K, D, deltas):
    dd = deltas[0]
    Yiv = [K*(1+rvec[i])*dd*phi_fun(uvec[i]) for i in [0,1]]
    bY = -K*dd*phi_fun(sum(uvec))
    eval_y = 1 + (sum(Yiv)+ bY)/W
    return(eval_y)

def Check_Stable(uvec,xvec,yvec,rvec,W,K,D,deltas,beta,gamma):
    # Given stable points and parameters, checks stability
    # returns 1 if stable, 0 if unstable
    # deltas are [delta_D, delta_K, delta_pc]
    Jacobian_UR = Jac_UR(uvec, rvec, K, D, W, deltas, beta, gamma)
    lambdas = np.zeros(7)
    lambdas[0:5] = LA.eigvals(Jacobian_UR) # this should be an array of 5 elements
    lambdas[5] = eval_x(uvec, rvec, W, D, deltas)
    lambdas[6] = eval_y(uvec, rvec, W, K, D, deltas)
    lambdas = np.absolute(np.real(lambdas))
    is_stable = int(sum(if_stable)<1)
    return(is_stable,lambdas)

def FindEquilibrium(uvec,xvec,yvec,rvec,W,K,pc,D,deltas,beta,gamma,tsteps= 10):
    # uvec,xvec,yvec,rvec have initial values
    # runs recursions until a stable point is reached
    # deltas are [delta_D, delta_K, delta_pc]
    reached_eq = 0
    prev = np.concatenate((uvec,xvec,yvec,rvec), axis = None)
    for t in range(0,tsteps): # could also do While reached_eq= 0, but could lead to infinite loop if no stable point
        ucurr, xcurr, ycurr, rcurr, Wcurr = NextGen(uvec,xvec,yvec,rvec, D, K,pc,deltas, beta, gamma)
        curr = np.concatenate((ucurr,xcurr,ycurr,rcurr), axis = None)
        if np.allclose(curr,prev): # checks that the two vectors are equal
            reached_eq = 1
            return(uvec, xvec, yvec, rvec, W, t, reached_eq)
        else:
            uvec,xvec,yvec,rvec,W = ucurr, xcurr, ycurr, rcurr, Wcurr

            prev = curr
    
    return(uvec,xvec,yvec,rvec,W,t,reached_eq) 
    
def Bif_justU(uvec,W,K,D,pc,beta,gamma):
    deltas = [0,0,0]; xvec = [0,0,0]; yvec = [0,0,0]
    rvec = [1,1] #r always starts out at 1
    W = 3*pc
    tsteps = 1000
    step = 0.2
    nsteps1 = 1.0/step + 1
    ncombos = nsteps1*(nsteps1-1)/2
    eq_data = np.zeros(ncombos,9)#equilibrium data. columns are u1,u2,bu,r1,r2,w,reached_eq,is_stable,t
    i = 0
    for u1 in np.linspace(0,1,nsteps1):
        nsteps2 = (1 - u1)/step + 1
        for u2 in np.linspace(0,1-u1,nsteps2):
            bu = 1 - u1 - u2
            eq_data[i,:] = FindStablePoint(uvec,xvec,yvec,rvec,W,K,pc,D,deltas,beta,gamma,tsteps)
            i += 1

In [135]:
# Special case K = 0
# just set D = 1

# initials:
mu = 0.2; sigma = 1 # These are the parameters of Wakano Aoki 2007
norm = scs.norm(mu,sigma**2)
d = 0
pc = pcfun(d, norm)
K = 0; D = 1;
deltad = 0.05; deltaD = 0.05
dk = Kfun(d+deltad,norm)-K
dpc = pcfun(d+deltad,norm)- pc
dd = 0.05
deltas = [dd, dk, dpc]
beta = gamma = 1 #for simplicity
uvec = [0,0,1]
rvec = [1,1]
xvec = yvec =  [0,0,0]
W = 3*pc
tsteps = 10000
uvec,xvec,yvec,rvec,W,tstop,reached_eq = FindEquilibrium(uvec,xvec,yvec,rvec,W,K,pc,D,deltas,beta,gamma, tsteps)
#FindEquilibrium(uvec,xvec,yvec,rvec,W,K,pc,D,deltas,beta,gamma, tsteps)
J_UR = Jac_UR(uvec, rvec, K, D, W, deltas, beta, gamma)
lambdas_u = LA.eigvals(J_UR)
print("The UR eigenvalues are %s" % np.real(lambdas_u))
print("The stable point is %s" %uvec, rvec)
eval_x(uvec,rvec,W,D,deltas)
d_to_try = np.linspace(0,5,100)
D_to_try = np.linspace(0,1,4)
dkdpc_vec = [[dd, Kfun(d+delta,norm) - K,pcfun(d+delta,norm) - pc]  for delta in d_to_try]

lambdasx = np.array([np.array([eval_x(uvec, rvec, W, D, deltas) for deltas in dkdpc_vec]) for D in D_to_try])



The UR eigenvalues are [0.         0.29272954 0.29272954 0.29272954 0.29272954]
The stable point is [0.41233055 0.41233055 0.17533889] [0.708061312738489, 0.708061312738489]


array([0.98674469, 0.98594864, 0.98515258, 0.98435652])

In [58]:
# TESTS

mu = 1
sigma = 1
norm = scs.norm(mu,sigma)
pc = pcfun(0,norm)

phi_fun(1/4)
p_i = 0.5; v = 1; r_i = 0.5; D = 0.2; K = 0.68; pc = 0.17
Wv_fun(p_i,v,r_i,D,K,pc)

p_1 = 0; p_2 = 0.5; v = 1;D = 0.2; K = 0.68; pc = 0.17
Wvbar_fun(p_1,p_2,v,D,K,pc)

r_i = 1; p_i = 0.5
ri_fun(r_i, p_i)

u1 = u2 = 0.3
bu = 1 - u1 -u2
uvec = [u1,u2,bu]
x1 = x2 = bx = y1 = y2 = by = 0
xvec = [x1,x2,bx]; yvec = [y1,y2,by]
r1 = r2 = 0.5
rvec = [r1,r2]

#TO-DO: Check NextGen more thoroughly
D = 0; K = 0.68; pc = 0.17
ans = NextGen(uvec,xvec,yvec,rvec,D,K,pc)
print("next gen" + str(sum(ans[0:3])))
print(ans)

theta_fun(0.5)

W = 0.5
Ai_jac(uvec,rvec,W,1)

phi_fun(1)

#TO-DO: test Bi_jac
deltas = [0, 0, 1]
dk = deltas[1]
dpc = deltas[2]
rvec = [1,1]
uvec = [0.5,0.5]
sum([rvec[i]*(dk*(uvec[i] + D*phi_fun(uvec[i]))+dpc) for i in [0,1]])

#TO-DO: test E_jac

#TO-DO: test U_jac

# TO-DO: Check eval_x
uvec = [0.3, 0.3, 0.1]
rvec = [0.7, 0.7]
W = 2
D = 0.2
deltas = [0.01,0.05,-0.02]
eval_x(uvec,rvec,W, D, deltas)

0
0.0
next gen[0.35551331 0.42015209 0.2243346 ]
(array([0.35551331, 0.42015209, 0.2243346 ]), array([0., 0., 0.]), array([0., 0., 0.]), [0.6166666666666667, 0.6])


0.982152

[-0.084, -0.084, -0.04800000000000002]

array([-0.23606798,  4.23606798])

[1, 1]

1
2
3


array([[0., 0.],
       [1., 2.],
       [0., 0.]])

1